# Suspicious GUP Usage
Detects execution of the Notepad++ updater in a suspicious directory, which is often used in DLL side-loading attacks

## Rule Content
```
- title: Suspicious GUP Usage
  id: 0a4f6091-223b-41f6-8743-f322ec84930b
  description: Detects execution of the Notepad++ updater in a suspicious directory,
    which is often used in DLL side-loading attacks
  status: experimental
  references:
  - https://www.fireeye.com/blog/threat-research/2018/09/apt10-targeting-japanese-corporations-using-updated-ttps.html
  tags:
  - attack.defense_evasion
  - attack.t1073
  author: Florian Roth
  date: 2019/02/06
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image: '*\GUP.exe'
    filter:
      Image:
      - C:\Users\\*\AppData\Local\Notepad++\updater\gup.exe
      - C:\Users\\*\AppData\Roaming\Notepad++\updater\gup.exe
      - C:\Program Files\Notepad++\updater\gup.exe
      - C:\Program Files (x86)\Notepad++\updater\gup.exe
    condition: selection and not filter
  falsepositives:
  - Execution of tools named GUP.exe and located in folders different than Notepad++\updater
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:*\\GUP.exe AND (NOT (process_path.keyword:(C\:\\Users\\*\\AppData\\Local\\Notepad\+\+\\updater\\gup.exe OR C\:\\Users\\*\\AppData\\Roaming\\Notepad\+\+\\updater\\gup.exe OR C\:\\Program\ Files\\Notepad\+\+\\updater\\gup.exe OR C\:\\Program\ Files\ \(x86\)\\Notepad\+\+\\updater\\gup.exe))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()